In [3]:
# %% [markdown]
# # Figure C — Ephemeris Growth: σ(Tₙ) Projection
# This notebook cell:
# - Projects σ(Tₙ) versus calendar date for each target (catalog vs optional "this work").
# - Reports percent reduction vs catalog at the end of the window.
# - Saves: 
#     - results/FigureC_sigma_projection.png  (figure)
#     - results/FigureC_sigma_projection.csv  (time series)
#     - results/FigureC_sigma_summary.csv     (end-of-window summary)

# %%
# --- Imports & setup (independent of TRICERATOPS) ---
%matplotlib inline

import os
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

os.makedirs("results", exist_ok=True)

# --- INPUTS (edit here) -------------------------------------------------------
# Periods (P) & epochs (T0) are in days (e.g., BTJD or JD—just be consistent).
# Uncertainties are 1σ in days.
# If you only have catalog uncertainties, leave the *_new fields as None.
targets = [
    # name, TIC,            P,           sigP_cat,   T0,            sigT0_cat,   sigP_new,    sigT0_new
    ("Target A", 119584412, 16.02749976, 0.000010,  1908.046283,    0.00150,     None,        None),
    ("Target B",  37749396, 13.47582381, 0.000006,  1392.306006,    0.00090,     None,        None),
    ("Target C", 311183180,  9.34853858, 0.000020,  2149.632747,    0.00120,     None,        None),
]
# Notes:
# - Replace sigP_cat/sigT0_cat with your catalog uncertainties.
# - If you have refined values from your fit, set sigP_new/sigT0_new (else keep None).

# Plot window (calendar dates)
start_date = dt.date.today()
years_span = 3.0
end_date   = start_date + dt.timedelta(days=int(round(365.25 * years_span)))
N_points   = 400  # resolution of the projection

# --- Core helpers -------------------------------------------------------------
def sigma_t_at_date(P, T0, sigP, sigT0, t):
    """σ(Tn) at epoch t (same units as T0), assuming uncorrelated P and T0."""
    n = (t - T0) / P
    return np.sqrt(sigT0**2 + (n**2) * sigP**2)

def build_calendar_grid(start_date, end_date, N):
    """Return pandas.DatetimeIndex (calendar) and Δdays relative to start."""
    cal = pd.date_range(start_date, end_date, periods=N)
    delta_days = (cal - cal[0]).days.astype(float)
    return cal, delta_days

# --- Compute projections ------------------------------------------------------
cal_dates, delta_days = build_calendar_grid(start_date, end_date, N_points)

# IMPORTANT:
# We don't actually need the *absolute* mapping between calendar and BTJD/JD.
# σ(Tn) growth depends on the *number of cycles since T0*, i.e., (t - T0)/P.
# We therefore construct an ephemeris time axis per target as: t_ephem = T0 + Δdays.
# This keeps units consistent and uses the real calendar only for x-axis labels.
rows = []
summary_rows = []

# Precompute all series first to set a common y-limit (prevents clipping).
series = []  # list of dicts: {name, tic, sig_cat_days, sig_new_days, pct_red}
max_hours = 0.0

for (name, tic, P, sigP_cat, T0, sigT0_cat, sigP_new, sigT0_new) in targets:
    t_ephem = T0 + delta_days
    sig_cat = sigma_t_at_date(P, T0, sigP_cat, sigT0_cat, t_ephem)   # days
    have_new = (sigP_new is not None) and (sigT0_new is not None)
    sig_new = sigma_t_at_date(P, T0, sigP_new, sigT0_new, t_ephem) if have_new else None

    cat_ref = float(sig_cat[-1])
    if have_new:
        new_ref = float(sig_new[-1])
        pct_red = (1.0 - new_ref / cat_ref) * 100.0 if np.isfinite(new_ref) and cat_ref > 0 else np.nan
        max_hours = max(max_hours, float(np.nanmax(sig_new) * 24.0))
    else:
        new_ref, pct_red = np.nan, np.nan

    max_hours = max(max_hours, float(np.nanmax(sig_cat) * 24.0))

    series.append(dict(
        name=name, tic=tic,
        sig_cat_days=sig_cat,
        sig_new_days=sig_new,
        have_new=have_new,
        pct_red=pct_red
    ))

# Make a flexible panel (1xN) figure
nT = len(targets)
fig, axs = plt.subplots(1, nT, figsize=(5.0*nT + 2, 4.2), sharey=True)
axs = np.atleast_1d(axs)

# Populate rows, summary, and plot
for ix, s in enumerate(series):
    name = s["name"]; tic = s["tic"]
    sig_cat = s["sig_cat_days"]; sig_new = s["sig_new_days"]; have_new = s["have_new"]; pct_red = s["pct_red"]

    for j in range(len(cal_dates)):
        rows.append({
            "name": name,
            "TIC": tic,
            "date": cal_dates[j].date().isoformat(),
            "sigma_cat_days": float(sig_cat[j]),
            "sigma_new_days": float(sig_new[j]) if have_new else np.nan
        })

    summary_rows.append({
        "name": name,
        "TIC": tic,
        "date_end": cal_dates[-1].date().isoformat(),
        "sigma_cat_end_days": float(sig_cat[-1]),
        "sigma_new_end_days": float(sig_new[-1]) if have_new else np.nan,
        "percent_reduction_vs_catalog": float(pct_red) if np.isfinite(pct_red) else np.nan
    })

    # ---- Plotting (σ in hours) ----
    ax = axs[ix]
    y_cat = sig_cat * 24.0
    ax.plot(cal_dates, y_cat, "-", lw=2, label="Catalog")
    if have_new:
        y_new = sig_new * 24.0
        ax.plot(cal_dates, y_new, "--", lw=2, label="This work")
        ax.legend(loc="upper left", frameon=False)
        # Annotate percent reduction
        ax.text(0.02, 0.95, f"Δ @ end ≈ {pct_red:.0f}%", transform=ax.transAxes,
                ha="left", va="top")

    # Small end-of-window label for the solid curve
    ax.text(0.98, 0.05, f"{y_cat[-1]:.3f} h", transform=ax.transAxes,
            ha="right", va="bottom", fontsize=9)

    ax.set_title(f"{name} (TIC {tic})")
    ax.set_xlabel("Date")
    if ix == 0:
        ax.set_ylabel(r"$\sigma(T_n)$  [hours]")
    ax.grid(True, alpha=0.3)

# Common y-limit so nothing clips
for ax in axs:
    ax.set_ylim(0, max_hours * 1.1)

plt.tight_layout()

# --- Save outputs -------------------------------------------------------------
ts_csv = "results/FigureC_sigma_projection.csv"
fig_png = "results/FigureC_sigma_projection.png"
sum_csv = "results/FigureC_sigma_summary.csv"

pd.DataFrame(rows).to_csv(ts_csv, index=False)
pd.DataFrame(summary_rows).to_csv(sum_csv, index=False)
plt.savefig(fig_png, dpi=200)
plt.close()

print("Saved:")
print(" -", ts_csv)
print(" -", sum_csv)
print(" -", fig_png)

# Show the summary table inline for quick review
pd.DataFrame(summary_rows)

Saved:
 - results/FigureC_sigma_projection.csv
 - results/FigureC_sigma_summary.csv
 - results/FigureC_sigma_projection.png


,name,TIC,date_end,sigma_cat_end_days,sigma_new_end_days,percent_reduction_vs_catalog
0,Target A,119584412,2028-10-02,0.001649,NaN,NaN
1,Target B,37749396,2028-10-02,0.001024,NaN,NaN
2,Target C,311183180,2028-10-02,0.002634,NaN,NaN


In [4]:
# === Supply refined ephemerides for Figure C (two ways) =======================
# Place this cell ABOVE the plotting cell. Run this first, then re-run the plot cell.

import numpy as np
import pandas as pd

# ----------------------------- PATH A: DIRECT VALUES --------------------------
# If you already have your refined uncertainties (1σ, in DAYS), fill them below.
# Leave any unknown "None" and use PATH B instead.
targets_override = [
    # name, TIC,            P,           sigP_cat,   T0,            sigT0_cat,   sigP_new,      sigT0_new
    ("Target A", 119584412, 16.02749976, 0.000010,   1908.046283,   0.00150,     None,          None),
    ("Target B",  37749396, 13.47582381, 0.000006,   1392.306006,   0.00090,     None,          None),
    ("Target C", 311183180,  9.34853858, 0.000020,   2149.632747,   0.00120,     None,          None),
]

# ----------------------- PATH B: BUILD FROM MID-TRANSIT TIMES -----------------
# If you have measured mid-transit times (in the SAME day scale as T0, e.g., BTJD),
# paste them here. Each entry: dict with 'name', 'tic', 'P', 'sigP_cat', 'T0', 'sigT0_cat',
# and a list of 't_mids' and 't_errs' (same length), and optionally the integer epochs 'n'.
# If 'n' is omitted, epochs will be inferred by rounding (t - T0)/P.
midtimes_input = [
    # Example (leave commented or replace with your real numbers):
    # {
    #   "name": "Target A", "tic": 119584412,
    #   "P": 16.02749976, "sigP_cat": 1.0e-5, "T0": 1908.046283, "sigT0_cat": 0.00150,
    #   "t_mids": [1908.04630, 1924.07385, 1940.10133],               # BTJD (example)
    #   "t_errs": [0.00080,    0.00085,    0.00090],                  # days (1σ)
    #   # "n":    [0, 1, 2],                                         # optional integer epochs
    # },
]

def _weighted_linear_ephemeris(t, sig, n=None, T0_ref=None):
    """
    Fit t ~ T0 + n*P with weights 1/sig^2.
    Returns: P, T0, sigP, sigT0 (1σ), cov_P_T0
    """
    t = np.asarray(t, float)
    sig = np.asarray(sig, float)
    w = 1.0 / np.asarray(sig, float)**2

    if n is None:
        # infer epochs by rounding around provided T0_ref (or first t)
        if T0_ref is None:
            T0_ref = t.min()
        # crude estimate of n with input cadence ~P; refine iteratively
        # start with zero-centered n:
        n_est = np.round((t - T0_ref) / (t[1]-t[0]) if len(t) > 1 else 1.0)
        n = n_est.astype(int)
    else:
        n = np.asarray(n, int)

    # Design matrix for [T0, P]
    X = np.vstack([np.ones_like(n, float), n.astype(float)]).T
    W = np.diag(w)
    XtW = X.T @ W
    cov = np.linalg.inv(XtW @ X)
    beta = cov @ (XtW @ t)
    T0_fit, P_fit = beta[0], beta[1]

    # Residual var (weighted)
    r = t - (T0_fit + n*P_fit)
    dof = max(1, len(t) - 2)
    s2 = (w * r**2).sum() / w.sum() * (len(t)/dof)  # scale by dof

    cov_scaled = cov * s2
    sigT0 = float(np.sqrt(cov_scaled[0,0]))
    sigP  = float(np.sqrt(cov_scaled[1,1]))
    cov_P_T0 = float(cov_scaled[0,1])
    return P_fit, T0_fit, sigP, sigT0, cov_P_T0

def build_targets_from_midtimes(mid_ins, base_targets):
    """Merge midtime-derived (sigP_new, sigT0_new) into the targets table."""
    # start from PATH A values (so we keep P, T0, catalog sigmas)
    df = pd.DataFrame(base_targets,
                      columns=["name","TIC","P","sigP_cat","T0","sigT0_cat","sigP_new","sigT0_new"])
    rows = []
    for _, row in df.iterrows():
        entry = {**row.to_dict()}
        # find matching midtime set
        mt = next((m for m in mid_ins if m["tic"] == entry["TIC"]), None)
        if mt is not None:
            Pfit, T0fit, sigP, sigT0, _ = _weighted_linear_ephemeris(
                mt["t_mids"], mt["t_errs"], mt.get("n", None), T0_ref=entry["T0"]
            )
            # Use the uncertainties from the fit; keep the published P/T0 values for consistency
            entry["sigP_new"]  = sigP
            entry["sigT0_new"] = sigT0
        rows.append(entry)
    out = [
        (r["name"], r["TIC"], r["P"], r["sigP_cat"], r["T0"], r["sigT0_cat"], r["sigP_new"], r["sigT0_new"])
        for r in rows
    ]
    return out

# Decide which path to use:
if len(midtimes_input) > 0:
    targets = build_targets_from_midtimes(midtimes_input, targets_override)
else:
    targets = targets_override

# Quick sanity print so you see what's being used:
print("Targets being used for Figure C (1σ days):")
for t in targets:
    print(f" - {t[0]} (TIC {t[1]}): sigP_cat={t[3]:.3g}, sigT0_cat={t[5]:.3g}, "
          f"sigP_new={t[6] if t[6] is not None else 'None'}, "
          f"sigT0_new={t[7] if t[7] is not None else 'None'}")

Targets being used for Figure C (1σ days):
 - Target A (TIC 119584412): sigP_cat=1e-05, sigT0_cat=0.0015, sigP_new=None, sigT0_new=None
 - Target B (TIC 37749396): sigP_cat=6e-06, sigT0_cat=0.0009, sigP_new=None, sigT0_new=None
 - Target C (TIC 311183180): sigP_cat=2e-05, sigT0_cat=0.0012, sigP_new=None, sigT0_new=None


In [5]:
# %% [markdown]
# # Figure C — Ephemeris Growth: σ(Tₙ) Projection (Final)
# - Projects σ(Tₙ) vs date for each target using catalog (and optional "this work") uncertainties.
# - Computes percent reduction vs catalog at end of window (defaults to 0% if no new values given).
# - Saves:
#     - results/FigureC_sigma_projection.png  (figure)
#     - results/FigureC_sigma_projection.csv  (time series)
#     - results/FigureC_sigma_summary.csv     (end-of-window summary)

# %%
%matplotlib inline
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
import datetime as dt

os.makedirs("results", exist_ok=True)

# -------------------- EDIT HERE (optional) --------------------
# If you ALREADY have your refined 1σ uncertainties (in DAYS), put them here.
# Leave a TIC out to use catalog-only for that target.
sigmas_new_by_tic = {
    # 119584412: dict(sigP_new=4.0e-06, sigT0_new=6.0e-04),
    # 37749396:  dict(sigP_new=3.0e-06, sigT0_new=4.0e-04),
    # 311183180: dict(sigP_new=8.0e-06, sigT0_new=5.0e-04),
}

# If you’d rather derive new uncertainties from your mid-times, fill this instead.
# (Leave empty to skip.)
midtimes_input = [
    # {
    #   "tic": 119584412,
    #   "P_guess": 16.02749976, "T0_guess": 1908.046283,
    #   "t_mids": [1908.04631, 1924.07379, 1940.10129],  # BTJD
    #   "t_errs": [0.00060,     0.00055,     0.00062],    # days
    # },
]

# If no new values are supplied, set True to report 0% (instead of NaN).
assume_no_improvement = True
# --------------------------------------------------------------

# Base target table (catalog numbers; 1σ in days)
targets = [
    # name, TIC,            P,           sigP_cat,   T0,            sigT0_cat
    ("Target A", 119584412, 16.02749976, 0.000010,   1908.046283,   0.00150),
    ("Target B",  37749396, 13.47582381, 0.000006,   1392.306006,   0.00090),
    ("Target C", 311183180,  9.34853858, 0.000020,   2149.632747,   0.00120),
]

# --- helper: weighted linear ephemeris fit (optional path from mid-times) ---
def fit_ephemeris_from_midtimes(P_guess, T0_guess, t_mids, t_errs):
    t_mids, t_errs = np.asarray(t_mids, float), np.asarray(t_errs, float)
    # Estimate integer epochs near guess and refit robustly
    n0 = np.rint((t_mids - T0_guess) / P_guess).astype(int)
    X = np.c_[np.ones_like(n0), n0]           # columns: [1, n]
    w = 1.0 / np.clip(t_errs, 1e-10, np.inf)**2
    XtWX = X.T @ (w[:, None] * X)
    XtWy = X.T @ (w * t_mids)
    beta = np.linalg.solve(XtWX, XtWy)        # [T0, P]
    cov  = np.linalg.inv(XtWX)
    T0_new, P_new = float(beta[0]), float(beta[1])
    sigT0_new, sigP_new = float(np.sqrt(cov[0,0])), float(np.sqrt(cov[1,1]))
    return T0_new, P_new, sigT0_new, sigP_new

# Collect any midtime-based updates into sigmas_new_by_tic
for item in midtimes_input:
    tic = int(item["tic"])
    T0n, Pn, sT0n, sPn = fit_ephemeris_from_midtimes(
        item["P_guess"], item["T0_guess"], item["t_mids"], item["t_errs"]
    )
    sigmas_new_by_tic[tic] = dict(sigP_new=sPn, sigT0_new=sT0n)

# --- projection math ---
def sigma_t_at_date(P, T0, sigP, sigT0, t):
    n = (t - T0) / P
    return np.sqrt(sigT0**2 + (n**2) * sigP**2)

# Calendar grid
start_date = dt.date.today()
years_span = 3.0
end_date   = start_date + dt.timedelta(days=int(round(365.25*years_span)))
cal_dates  = pd.date_range(start_date, end_date, periods=400)
delta_days = (cal_dates - cal_dates[0]).days.astype(float)

# Build figure + outputs
rows, summary = [], []
fig, axs = plt.subplots(1, len(targets), figsize=(5.2*len(targets)+1.5, 4.2), sharey=True)
axs = np.atleast_1d(axs)

for ax, (name, tic, P, sigP_cat, T0, sigT0_cat) in zip(axs, targets):
    # New uncertainties if provided
    new = sigmas_new_by_tic.get(int(tic), None)
    have_new = new is not None and all(k in new for k in ("sigP_new","sigT0_new"))
    sigP_new = new["sigP_new"] if have_new else None
    sigT0_new = new["sigT0_new"] if have_new else None

    # Time axis in ephemeris units
    t_ephem = T0 + delta_days
    sig_cat = sigma_t_at_date(P, T0, sigP_cat, sigT0_cat, t_ephem)
    if have_new:
        sig_new = sigma_t_at_date(P, T0, sigP_new, sigT0_new, t_ephem)
    else:
        sig_new = None

    # End-of-window metrics
    cat_ref = float(sig_cat[-1])
    if have_new:
        new_ref = float(sig_new[-1])
        pct_red = (1 - new_ref/cat_ref) * 100.0 if cat_ref > 0 else np.nan
    else:
        new_ref = cat_ref if assume_no_improvement else np.nan
        pct_red = 0.0 if assume_no_improvement else np.nan

    # Save rows
    for j in range(len(cal_dates)):
        rows.append(dict(name=name, TIC=tic,
                         date=cal_dates[j].date().isoformat(),
                         sigma_cat_days=float(sig_cat[j]),
                         sigma_new_days=float(sig_new[j]) if sig_new is not None else np.nan))
    summary.append(dict(name=name, TIC=tic,
                        date_end=cal_dates[-1].date().isoformat(),
                        sigma_cat_end_days=cat_ref,
                        sigma_new_end_days=new_ref,
                        percent_reduction_vs_catalog=pct_red))

    # Plot (σ in hours)
    ax.plot(cal_dates, sig_cat*24.0, "-", lw=2, label="Catalog")
    if have_new:
        ax.plot(cal_dates, sig_new*24.0, "--", lw=2, label="This work")
        ax.legend(loc="upper left", frameon=False)
        ax.text(0.02, 0.95, f"Δ @ end ≈ {pct_red:.0f}%", transform=ax.transAxes,
                ha="left", va="top")
    # annotate end-of-window value
    ax.text(0.98, 0.05, f"{(new_ref if have_new else cat_ref)*24.0:.3f} h",
            transform=ax.transAxes, ha="right", va="bottom", color="0.3")
    ax.set_title(f"{name} (TIC {tic})")
    ax.set_xlabel("Date")
    ax.grid(True, alpha=0.3)
    ax.set_ylim(bottom=0.0)

axs[0].set_ylabel(r"$\sigma(T_n)$  [hours]")
plt.tight_layout()

# Write outputs
ts_csv = "results/FigureC_sigma_projection.csv"
sum_csv = "results/FigureC_sigma_summary.csv"
fig_png = "results/FigureC_sigma_projection.png"
pd.DataFrame(rows).to_csv(ts_csv, index=False)
pd.DataFrame(summary).to_csv(sum_csv, index=False)
plt.savefig(fig_png, dpi=200)
plt.close()

print("Saved:\n -", ts_csv, "\n -", sum_csv, "\n -", fig_png)
print("\nTargets being used for Figure C (1σ days):")
for (name, tic, P, sigP_cat, T0, sigT0_cat) in targets:
    new = sigmas_new_by_tic.get(int(tic), {})
    print(f" - {name} (TIC {tic}): sigP_cat={sigP_cat}, sigT0_cat={sigT0_cat}, "
          f"sigP_new={new.get('sigP_new')}, sigT0_new={new.get('sigT0_new')}")

# Show summary inline
pd.DataFrame(summary)

Saved:
 - results/FigureC_sigma_projection.csv 
 - results/FigureC_sigma_summary.csv 
 - results/FigureC_sigma_projection.png

Targets being used for Figure C (1σ days):
 - Target A (TIC 119584412): sigP_cat=1e-05, sigT0_cat=0.0015, sigP_new=None, sigT0_new=None
 - Target B (TIC 37749396): sigP_cat=6e-06, sigT0_cat=0.0009, sigP_new=None, sigT0_new=None
 - Target C (TIC 311183180): sigP_cat=2e-05, sigT0_cat=0.0012, sigP_new=None, sigT0_new=None


,name,TIC,date_end,sigma_cat_end_days,sigma_new_end_days,percent_reduction_vs_catalog
0,Target A,119584412,2028-10-03,0.001649,0.001649,0.0
1,Target B,37749396,2028-10-03,0.001024,0.001024,0.0
2,Target C,311183180,2028-10-03,0.002634,0.002634,0.0
